<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/grandes_modelos_de_lenguaje/llms_con_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Largos de Lenguaje (Large Language Models)

Un LLMs es un tipo de modelo de inteligencia artificial diseñado para comprender y generar texto en lenguaje natural de manera altamente sofisticada. Estos modelos están entrenados en grandes cantidades de datos textuales y utilizan técnicas de aprendizaje automático, como el aprendizaje profundo, para capturar patrones y estructuras del lenguaje.

Los grandes modelos de lenguaje han revolucionado el campo del procesamiento de lenguaje natural debido a su capacidad para comprender y generar texto con un nivel de sofisticación cada vez mayor. Estos modelos son útiles en una variedad de aplicaciones, incluyendo chatbots, asistentes virtuales, generación de contenido, análisis de sentimientos, resumen de texto y mucho más. Además, los grandes modelos de lenguaje también se utilizan como base para modelos más especializados y adaptados a tareas específicas.

Es importante destacar que el entrenamiento y la implementación de grandes modelos de lenguaje requieren una infraestructura computacional potente debido a su complejidad y requerimientos de recursos. Sin embargo, estos modelos están impulsando avances significativos en el procesamiento de lenguaje natural y abriendo nuevas posibilidades en la interacción hombre-máquina y la generación de texto de alta calidad.

### Introducción a LangChain

LangChain es un marco creado para ayudarlo a crear aplicaciones basadas en LLM más fácilmente al proporcionarle las siguientes capacidades:

* Una interfaz genérica para una variedad de diferentes **Modelos Largos de Lenguaje**
* Un marco para ayudar a administrar **Promtps**
* Una interfaz central para gestionar la **Memoria** a largo plazo, datos externos con **Índices**, otros LLM con **Cadenas** y otros **Agentes** para tareas que un LLM no puede manejar (por ejemplo, cálculos o búsqueda).

---

**Objetivos** Revisión de los siguientes conceptos:

- LLMs (Grandes modelos de lenguaje)
- Prompt Templates (Plantillas)
- Chains (Cadenas)
- Agents and Tools (Agentes y herramientas)
- Memory (Memoria)
- Document Loaders (Cargadores de documentos)
- Indexes (Índices)

![Vector Search](https://images.idgesg.net/images/article/2021/09/bing-ann-vector-search-architecture-100904677-large.jpg?auto=webp&quality=85,70)

### Instalación

In [1]:
!pip install langchain huggingface_hub openai wikipedia sentence_transformers faiss-cpu unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.6/934.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## 1. LLMs

Una interfaz genérica para todos los LLM. Ver todos los proveedores de LLM: https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [2]:
import os
import sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/Inteligencia Artificial/Grandes Modelos de Lenguaje'
sys.path.append(path)

Hugging Face API key: https://huggingface.co/settings/tokens

In [5]:
from config import api_key_hgface

os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_key_hgface

In [6]:
from langchain import HuggingFaceHub

Enlace al modelo *flan-T5* de Google: https://huggingface.co/google/flan-t5-xxl

In [26]:
llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.1, "max_length":64})
text = "Traducir de español a inglés: Alrededor del mundo"
print(llm(text))

Around the world


OpenAI API key: https://platform.openai.com/account/api-keys

In [20]:
from config import api_key_openai

os.environ["OPENAI_API_KEY"] = api_key_openai

In [28]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [30]:
llm = OpenAI(temperature=0.1, model_name="text-davinci-003")
text = "¿Cuál sería un buen nombre para una startup disruptiva que ofrece servicios basados en inteligencia artificial?"
print(llm(text))



BrainBoost AI.


Lista de modelos disponibilizados por OpenAI: https://platform.openai.com/docs/models

In [31]:
llm = OpenAI(temperature=0.3, model_name="gpt-3.5-turbo")
text = "¿Cuál sería un buen nombre en español para una startup disruptiva que ofrece servicios basados en inteligencia artificial?"
print(llm(text))

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Algunas opciones podrían ser:

1. InteligenciaX
2. AI Solutions
3. IA Innovación
4. Neuralia
5. Cognitiva
6. InteligenciaLab
7. IA Avanzada
8. BrainTech
9. Inteligencia360
10. IA Futura


Costos de modelos ofertados por Open AI: https://openai.com/pricing

## 2. Prompt Templates

LangChain facilita la gestión y la rápida optimización.

Normalmente, cuando usa un LLM en una aplicación, no está enviando la entrada del usuario directamente al LLM. En su lugar, debe tomar la entrada del usuario y construir un aviso, y solo luego enviarlo al LLM.

In [32]:
llm("¿Puede Andrés Manuel López Obrador tener una conversación con Michael Jackson?")

'Como inteligencia artificial, no puedo determinar si una conversación entre Andrés Manuel López Obrador y Michael Jackson es posible, ya que Michael Jackson falleció en 2009. Sin embargo, si se tratara de una conversación hipotética en un contexto ficticio, podría ser posible en una obra de ficción o en una simulación virtual.'

In [33]:
prompt = """Question: ¿Puede Andrés Manuel López Obrador tener una conversación con Michael Jackson?

Pensemos paso a paso.

Respuesta: """
llm(prompt)

'No, ya que Michael Jackson falleció en 2009 y no es posible tener una conversación con alguien que ha fallecido.'

In [34]:
from langchain import PromptTemplate

template = """Pregunta: {pregunta}

Pensemos paso a paso.

Respuesta: """

prompt = PromptTemplate(template=template, input_variables=["pregunta"])

In [35]:
prompt.format(pregunta="¿Puede Andrés Manuel López Obrador tener una conversación con Michale Jackson?")

'Pregunta: ¿Puede Andrés Manuel López Obrador tener una conversación con Michale Jackson?\n\nPensemos paso a paso.\n\nRespuesta: '

In [37]:
try:
    llm(prompt)
except Exception as e:
    print("Requerimos de una cadena (Chain)")

Requerimos de una cadena (Chain)


## 3. Chains

Combina LLM y prompts en flujos de trabajo de varios pasos.

In [38]:
from langchain import LLMChain

In [39]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "¿Puede Andrés Manuel López Obrador tener una conversación con Michael Jackson?"
print(llm_chain.run(question))

No, Andrés Manuel López Obrador no puede tener una conversación con Michael Jackson ya que el cantante falleció en 2009. Sería imposible tener una conversación con alguien que ya no está vivo.


## 4. Agentes y Herramientas

Los agentes involucran a un LLM que toma decisiones sobre qué acciones tomar, ejecutar esa acción, tener en cuenta una observación y repetirla hasta que se termine.

Cuando se usan correctamente, los agentes pueden ser extremadamente poderosos.Para cargar agentes se deben comprender los siguientes conceptos:

- Herramienta (Tool): Es una función que debe realizar el agente. Esto puede ser por ejemplo: una búsqueda en Google, búsqueda en base de datos, ejecutar Python REPL (intérprete de python), entre otras cadenas.
- LLM: El modelo de lenguaje que impulsa al agente.

In [40]:
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [41]:
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [42]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [43]:
agent.run("¿En qué año se estrenó la película Inception con Leonardo Dicaprio? ¿Cuál sería este año elevado a la potencia 0,43?")



> Entering new AgentExecutor chain...
 I need to find out the year the movie was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Inception (2010 film)

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Inception
Summary: Inception is a 2010 science fiction action film written and directed by Christopher Nolan, who also produced the film with Emma Thomas, his wife. The film stars Leonardo DiCaprio as a professional thief who steals information by infiltrating the subconscious of his targets. He is offered a chance to have his criminal history erased as payment for the implantation of another person's idea into a target's subconscious. The ensemble cast includes Ken Watanabe, Joseph Gordon-Levitt, Marion Cotillard, Elliot Page, Tom Hardy, Dileep Rao, Cillian Murphy, Tom Berenger, and Michael Caine.
After the 2002 completion of Insomnia, Nolan presented to Warner Bros. a written 80-page treatment for a horror film envisioning "dream stealers," based on lucid dreaming. Deciding he needed more experience before tackling a production of this magnitude and complexity, Nolan shelved the project and instead worked on 2005's Batman Begins, 2006's The Prestige, and The Dark 

'El año 2010 elevado a la potencia 0,43 es 26.3253591394933.'

## 5. Memoria

Agregar Estado a Cadenas y Agentes.

La memoria es el concepto de estado persistente entre llamadas de una cadena/agente. LangChain proporciona una interfaz estándar para la memoria, una colección de implementaciones de memoria y ejemplos de cadenas/agentes que usan memoria.

In [44]:
from langchain import OpenAI, ConversationChain

In [45]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hola!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola!
AI:

> Finished chain.


' Hola! ¿Cómo estás? (Hello! How are you?)'

In [46]:
conversation.predict(input="¿Podemos hablar de IA?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? (Hello! How are you?)
Human: ¿Podemos hablar de IA?
AI:

> Finished chain.


' ¡Por supuesto! Me encantaría hablar de Inteligencia Artificial. ¿Qué quieres saber? (Of course! I would love to talk about Artificial Intelligence. What do you want to know?)'

In [47]:
conversation.predict(input="¿Qué me puedes decir de la inteligencia artificial generativa?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? (Hello! How are you?)
Human: ¿Podemos hablar de IA?
AI:  ¡Por supuesto! Me encantaría hablar de Inteligencia Artificial. ¿Qué quieres saber? (Of course! I would love to talk about Artificial Intelligence. What do you want to know?)
Human: ¿Qué me puedes decir de la inteligencia artificial generativa?
AI:

> Finished chain.


' La Inteligencia Artificial Generativa es una forma de Inteligencia Artificial que se centra en la creación de contenido nuevo y original. Esta tecnología se utiliza para generar contenido como imágenes, texto, audio y video. Esta tecnología se puede utilizar para crear contenido único y personalizado para los usuarios.'

## 6. Cargadores de documentos

La combinación de modelos de lenguaje con sus propios datos de texto es una forma poderosa de diferenciarlos. El primer paso para hacer esto es cargar los datos en "documentos", una forma elegante de decir algunos fragmentos de texto. Este módulo tiene como objetivo hacer esto fácil.

https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [48]:
from langchain.document_loaders import OnlinePDFLoader

Enlace al *paper: "Attention is all you need", Ashish Vaswani, 2017*: https://arxiv.org/pdf/1706.03762.pdf 

In [49]:
loader = OnlinePDFLoader("https://arxiv.org/pdf/1706.03762.pdf")
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [50]:
print (f'Existen {len(data)} documento (s) en el archivo')
print (f'Hay {len(data[0].page_content)} caracteres en el documento')

Existen 1 documento (s) en el archivo
Hay 38236 caracteres en el documento


## 7. Índices

Los índices se refieren a formas de estructurar documentos para que los LLM puedan interactuar mejor con ellos. Este módulo contiene funciones de utilidad para trabajar con documentos.

- Incrustaciones: Una incrustación es una representación numérica de una información, por ejemplo, texto, documentos, imágenes, audio, etc.
- Divisores de texto: Cuando desee manejar fragmentos largos de texto, es necesario dividir ese texto en fragmentos.
- Almacenes de vectores: Las bases de datos vectoriales almacenan e indexan incrustaciones de vectores de modelos NLP para comprender el significado y el contexto de cadenas de texto, oraciones y documentos completos para obtener resultados de búsqueda más precisos y relevantes.

In [54]:
from langchain.text_splitter import CharacterTextSplitter

In [57]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [58]:
from langchain.embeddings import HuggingFaceEmbeddings

In [59]:
embeddings = HuggingFaceEmbeddings()

Almacen de vectores: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html

In [60]:
from langchain.vectorstores import FAISS

In [65]:
db = FAISS.from_documents(docs, embeddings)

query = "¿What is Attention?"
docs = db.similarity_search(query)

In [69]:
print(docs[0].page_content)

To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9].

3 Model Architecture

Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35]. Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next.

The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers 

In [70]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9].

3 Model Architecture

Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35]. Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next.

The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers 

**Referencias**

* https://github.com/hwchase17/langchain
* https://huggingface.co/google/flan-t5-xl
* https://python.langchain.com/en/latest/index.html
* https://python.langchain.com/en/latest/modules/agents/tools.html
* https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html
* https://towardsdatascience.com/getting-started-with-langchain-a-beginners-guide-to-building-llm-powered-applications-95fc8898732c
